# Fast Async Code with Cython and AsyncIO

_by Stefan Behnel & Anton Caceres_

Stefan  (<stb@skoobe.de>)
-  Python developer since 2002
-  Cython core developer, main developer of lxml

Anton  (<ac@skoobe.de>)
-  Python developer since 2010
-  Organiser of PyCon-DE 2016 (this October!)

Skoobe
-------

* eBook subscription service
* 180 000+ books for 9.99€ / month
* Android, iOS apps - 5 stars
* Python based backend

![Alt text](https://staging-cdn.skoobe.net/static/v/12a6f0753eb3c5edbda7861ad83d6593873f4a2a244fc452f5e5d7cf82cb186e.jpg)

## What to expect from this talk

* Introduction to AsyncIO
* Introduction to Cython
* Async Cython coroutines
* Practical examples

AsyncIO
-------

* Module for writing async code with high concurrency
* Infrastructure for writing single-threaded concurrent code using coroutines
* Runs on Python 3.3+, backport to Py2 available

### Features:

* event loop with system-specific implementations
* Future class, similar to concurrent.futures
* coroutines and tasks based on yield from

### Coroutines

* based on generators using yield from (PEP 380)
* marked by @asyncio.coroutine or `async def` syntax (PEP 492)
* can be suspended, allowing the event loop to get on with other things

## AsyncIO in 5 Minutes

Cython
------

* Most widely used static Python compiler
* Outputs C code using the CPython C-API
* Extends Python language for optimisation and C/C++ interaction
* Open-source: cython.org / github

### Features:
* static code optimisation
* fast extensions for CPython (2.6 - 3.6+)
* widely used in Scientific Python ecosystem

-  Cython syntax generally follows Python 2.7

   -  plus all from Py3 that doesn't conflict

-  Switch to full Python 3 syntax/semantics by directive:

   `# cython: language_level=3`

-  Supports PEP 492 coroutines with async/await

   -  slower in Py2.6/7 than in Py3

## Cython in 10 Minutes

# Async Cython Coroutines with Python 3.5

In [ ]:
%load_ext cython

In [ ]:
import asyncio

In [ ]:
def run_async(coro):
    loop = asyncio.get_event_loop()
    result = loop.run_until_complete(coro)
    return result

## Cython coroutines are fully compatible with asyncio and Python coroutines

In [ ]:
%%cython
# cython: language_level = 3

async def add_one(fut):
    result = await fut
    print("Cy-ADD:", result)
    return result + 1

async def one():
    return 1

In [ ]:
run_async(add_one(add_one(one())))

In [ ]:
async def py_add_one(fut):
    result = await fut
    print("Py-ADD:", result)
    return result + 1

In [ ]:
run_async(add_one(py_add_one(one())))

## Let's play some ping-pong
-  one async Python function prints "ping"
-  one async Cython function prints "pong"
-  jump between the two

In [ ]:
%%cython

# example coro_map: {0: cy_dec1, 1: cy_dec1}

async def cy_dec1(coro_map, value, show=False):
    if show:
        print('pong')
    if value > 0:
        value = await coro_map[value % 2](coro_map, value - 1, show)
    return value

In [ ]:
# example coro_map: {0: py_dec1, 1: py_dec1}

async def py_dec1(coro_map, value, show=False):
    if show:
        print('ping')
    if value > 0:
        value = await coro_map[value % 2](coro_map, value - 1, show)
    return value

In [ ]:
run_async(py_dec1({0: cy_dec1, 1: py_dec1}, 6, show=True))

## How fast are Cython coroutines?

In [ ]:
%timeit run_async(py_dec1({0: py_dec1, 1: py_dec1}, 600)) # python-only

In [ ]:
%timeit run_async(cy_dec1({0: cy_dec1, 1: cy_dec1}, 600)) # cython-only

In [ ]:
%timeit run_async(py_dec1({0: cy_dec1, 1: py_dec1}, 600)) # alternate

## -> explain why I/O applications can reduce latency with faster processing intervals

## -> show an example where we receive little data and do a lot of processing on it

# Practical Example: Decryption Server

### Simple setup where client sends encrypted data stream to the server

![Alt text](http://i.imgur.com/8NB5zVe.jpg)

## _Decryption Server: code_
_(another notebook)_

## Thank you!

### Questions?

_Stefan: stb@skoobe.de, Anton: ac@skoobe.de_